This notebook leverages the `pyiron_ising` module to perform size scaling tests for various mutation algorithms. 
Key parameters can be adjusted in the all-caps header.

In [ ]:
QUEUE = 'cm'  # Remote queue to run on, set to None to use your local machine
CORES = 40  # How many cores to use, i.e. independent runs for each tests to get statistics
RUNTIME = 60*60*12  # 12 hrs in seconds, only used if QUEUE is not None
MAXSTEPS = int(1E6)  # The max number of iterations to use
# Dropping MAXSTEPS will allow you to run the calculations more quickly,
# But you may need to go through and shrink what sizes are used, otherwise many will fail to find a solution
CLEAN = True  # Delete all existing jobs and run from scratch
PLOT = False  # Whether to produce images from the data

# Setup

In [ ]:
from pyiron_ising import Project, Model, Mutation
from typing import Type, List, Callable, Union
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')
from functools import cached_property
from abc import ABC, abstractmethod


class Scaling(ABC):
    def __init__(
            self,
            project: Project,
            name: str,
            reps: List[int],
            min_like_neighbors=None,
            max_like_neighbors=None,
            n_steps: int = MAXSTEPS,
            n_print: int = MAXSTEPS,
            log_mutations: bool = False,
            cores: int = CORES if CORES is not None else 1,
            queue: Union[str, None] = QUEUE,
            run_time: Union[int, None] = RUNTIME,
            delete_existing_job: bool = False
    ):
        self.project = project
        self.name = name
        self.min_like_neighbors = min_like_neighbors
        self.max_like_neighbors = max_like_neighbors
        self._mutations = None

        self.jobs = []
        size = []
        for r in reps:
            ref = project.ising.job.Ising(f'{name}_ref_{r}', delete_existing_job=delete_existing_job)
            ref.input.model = self.model(repetitions=r, n_spins=2)
            for m in self.mutations:
                ref.input.mutations.add(m)
            ref.input.stopping_fitness = self.optimal_fitness_function(r)
            ref.input.n_steps = n_steps
            ref.input.n_print = n_print
            ref.input.log_mutations = log_mutations

            job = project.ising.job.ParallelIsing(f'{name}_{r}', delete_existing_job=delete_existing_job)
            job.ref_job = ref
            job.server.cores = cores
            if queue is not None:
                job.server.queue = queue
                if run_time is not None:
                    job.server.run_time = run_time
            self.jobs.append(job)
            size.append(len(ref.input.model))

        self.size = np.array(size)
        self.reps = np.array(reps)

    @property
    @abstractmethod
    def model(self) -> Type[Model]:
        pass

    @property
    @abstractmethod
    def mutations(self) -> List[Mutation]:
        pass

    @abstractmethod
    def optimal_fitness_function(self) -> Callable:
        pass

    def run(self) -> None:
        for j in self.jobs:
            j.run()

    @cached_property
    def name_mask(self):
        return np.any([self.project.job_table().job.values == j.name for j in self.jobs], axis=0)

    @cached_property
    def run_time(self):
        """Actual compute time for the job"""
        return self.project.job_table().totalcputime.values[self.name_mask]

    @cached_property
    def run_steps(self):
        """Median steps (to either solution or step limit)"""
        return np.array([job.postprocessing.median_end_frame for job in self.jobs])

    @cached_property
    def run_confidence(self):
        """95% confidence interval for the *mean* steps (assuming they're gaussian anyhow)"""
        return np.array([2.96 * job.postprocessing.end_frame.std() for job in self.jobs])

    def _figax(self, ax):
        if ax is None:
            fig, ax = plt.subplots()
        else:
            fig = None
        return fig, ax

    def plot(
        self,
        ax=None,
        xlabel="System size", 
        steps_ylabel="Median steps to solution",
        show_time=True, 
        time_ylabel="Run time",
        label='steps',
        marker='x',
        linestyle='none',
        **errorbar_kwargs
            
    ):
        fig, ax = self._figax(ax)

        ax.errorbar(
            self.size,
            self.run_steps,
            yerr=self.run_confidence,
            linestyle=linestyle,
            marker=marker,
            label=label,
            **errorbar_kwargs
        )
        ax.set_xlabel(xlabel)
        ax.set_ylabel(steps_ylabel)

        if show_time:
            time_ax = ax.twinx()
            time_ax.scatter(self.size, self.run_time, color='r', marker='+', label='time')
            time_ax.set_ylabel(time_ylabel)
        return fig, ax

    def plot_normalized(
        self,
        ax=None,
        xlabel="System size", ylabel="Normalized median steps",
        function=None,
        show_legend=True,
        **scatter_kwargs
    ):
        fig, ax = self._figax(ax)
        function = function if function is not None else lambda x: x

        ax.scatter(
            self.size,
            self.run_steps / function(self.size),
            **scatter_kwargs
        )
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)

        if show_legend:
            ax.legend()
        return fig, ax


def _optimal_score(n_tot, n_border, border_score):
    n_interior = n_tot - n_border
    return (n_interior + border_score * n_border) / n_tot


class Chain(ABC):
    @property
    def model(self):
        return self.project.ising.model.Chain1D

    @staticmethod
    def optimal_fitness_function(reps):
        return _optimal_score(reps, 4, (1 - 1) / 2)


class Square(ABC):
    @property
    def model(self):
        return self.project.ising.model.Square2D

    @staticmethod
    def optimal_fitness_function(reps):
        return _optimal_score(reps ** 2, 4 * reps, ((3 - 1) / 4))


class Hex(ABC):
    @property
    def model(self):
        return self.project.ising.model.Hex2D

    @staticmethod
    def optimal_fitness_function(reps):
        return _optimal_score(2 * reps ** 2, 4 * reps, ((4 - 2) / 6))


class BCC(ABC):
    @property
    def model(self):
        return self.project.ising.model.BCC3D

    @staticmethod
    def optimal_fitness_function(reps):
        return _optimal_score(2 * reps ** 3, 4 * reps ** 2, (4 - 4) / 8)


class Swapper(ABC):
    @property
    def mutations(self):
        if self._mutations is None:
            self._mutations = [self.project.ising.mutation.Swap()]
        return self._mutations


class Clusterer(ABC):
    @property
    def mutations(self):
        if self._mutations is None:
            self._mutations = [
                self.project.ising.mutation.Cluster(min_like_neighbors=self.min_like_neighbors),
                self.project.ising.mutation.Cluster(max_like_neighbors=self.max_like_neighbors)
            ]
        return self._mutations

In [ ]:
pr = Project('scaling')
if CLEAN:
    pr.remove_jobs_silently(recursive=True)

# 1D Chain

In [ ]:
class ChainSwap(Swapper, Chain, Scaling):
    pass


class ChainCluster(Clusterer, Chain, Scaling):
    pass



chain_swap = ChainSwap(
    pr, 
    'chain_swap', 
    [16, 32, 48, 64, 80, 96, 112, 128],
)

chain_cluster = ChainCluster(
    pr, 
    'chain_cluster', 
    [16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768],
)

In [ ]:
chain_swap.run()
chain_cluster.run()

In [ ]:
if PLOT:
    
    chain_swap.plot()
    plt.show()
    chain_cluster.plot()
    plt.show()
    
    def theorem1(n):
        return (n**4 / 501) + (n**2 * np.log(n) / 2)
    theorem1_label = "$\\frac{n^4}{501} + \\frac{n^2 \\ln n}{2}$"

    def theorem5(n):
        return n**(4/3)
    theorem5_label = "$n^{4/3}$"

    chain_swap.plot_normalized(function=theorem1, label=theorem1_label)
    plt.show()

    fig, ax = plt.subplots()
    chain_cluster.plot_normalized(ax=ax, function=None, label='$n$', show_legend=False)
    chain_cluster.plot_normalized(ax=ax, function=theorem5, label=theorem5_label, show_legend=True, marker='s')
    plt.show()

# 2D Square

In [ ]:
class SquareSwap(Swapper, Square, Scaling):
    pass


class SquareCluster(Clusterer, Square, Scaling):
    pass


square_swap = SquareCluster(
    pr, 
    'squre_swap',
    [4, 6, 8, 10, 12],
)

square_clusters = [
    SquareCluster(
        pr, 
        f'square_cluster{i}',
        [4, 8, 16, 32, 48, 64, 96, 128],
        min_like_neighbors=i,
        max_like_neighbors=i,
    )
    for i in [2, 3]
]

In [ ]:
square_swap.run()
for s in square_clusters:
    s.run()

In [ ]:
if PLOT:
    square_swap.plot()
    plt.show()
    for s in square_clusters:
        s.plot()
        plt.show()
        
    def logn(n):
        return np.log(n)

    def nlogn(n):
        return n * np.log(n)

    def nsq(n):
        return n**2

    square_swap.plot_normalized(function=nsq)
    plt.show()

    fig, ax = plt.subplots()
    markers=['x', '+', 'v', '^', '>', '<']
    for i, s in enumerate(square_clusters):
        s.plot_normalized(ax=ax, function=None, label=i+2, marker=markers[i])
    plt.show();

# 2D Hex

In [ ]:
class HexSwap(Swapper, Hex, Scaling):
    pass


class HexCluster(Clusterer, Hex, Scaling):
    pass

hex_swap = HexSwap(
    pr, 
    'hex_swap',
    [4, 6, 8, 10, 12],
)

hex_clusters = [
    HexCluster(
        pr, 
        f'hex_cluster{i}',
        [4, 8, 16, 32, 48, 64, 96, 128],
        min_like_neighbors=i,
        max_like_neighbors=i,
    )
    for i in [2, 3, 4, 5]
]

In [ ]:
hex_swap.run()
for h in hex_clusters:
    h.run()

In [ ]:
if PLOT:
    hex_swap.plot()
    plt.show()
    for h in hex_clusters:
        h.plot()
        plt.show()
        
    hex_swap.plot_normalized(function=None)
    plt.show()

    fig, ax = plt.subplots()
    for i, h in enumerate(hex_clusters):
        h.plot_normalized(ax=ax, function=None, label=i+2)
    plt.show();

# 3D BCC

In [ ]:
class BCCSwap(Swapper, BCC, Scaling):
    pass


class BCCCluster(Clusterer, BCC, Scaling):
    pass

bcc_swap = BCCSwap(
    pr, 
    'bcc_swap',
    [2, 3, 4, 5],
)

bcc_clusters = [
    BCCCluster(
        pr, 
        f'bcc_cluster{i}',
        [2, 4, 8, 16, 24, 32],
        min_like_neighbors=i,
        max_like_neighbors=i,
    )
    for i in [3, 4, 5, 6]
]

In [ ]:
bcc_swap.run()
for b in bcc_clusters:
    b.run()

In [ ]:
if PLOT:
    bcc_swap.plot()
    plt.show()
    for b in bcc_clusters:
        b.plot()
        plt.show()
        
    bcc_swap.plot_normalized(function=None)
    plt.show()

    fig, ax = plt.subplots()
    for i, b in enumerate(bcc_clusters):
        b.plot_normalized(ax=ax, function=None, label=i+3)
    plt.show();